In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(facecolor='white', frameon=False)

In [2]:
from pangeapy.cell import CellAnnotator
from pangeapy.meta import MetaAnnotator

In [3]:
cell_anno = CellAnnotator()
meta_anno = MetaAnnotator()

Checking-up download status of anno_models
Checking-up download status of meta_models


In [12]:
adata = sc.read_10x_h5(
    "/nfs/SCMGL/smc00_ref/2025_Craig-Schapiro_NatComm_islet/GSM6856687_Islet_EC_filtered_feature_bc_matrix.h5"
)

reading /nfs/SCMGL/smc00_ref/2025_Craig-Schapiro_NatComm_islet/GSM6856687_Islet_EC_filtered_feature_bc_matrix.h5
 (0:00:01)


In [13]:
adata

AnnData object with n_obs × n_vars = 11093 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [14]:
print(adata.X.min(), adata.X.max())

0.0 34000.0


In [15]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

adata = adata[
    (adata.obs.n_genes_by_counts < 2500) & 
    (adata.obs.n_genes_by_counts > 200) & 
    (adata.obs.pct_counts_mt < 5),
    :,
].copy()
adata.layers["counts"] = adata.X.copy()

filtered out 11 cells that have less than 200 genes expressed
filtered out 15167 genes that are detected in less than 3 cells


In [17]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

normalizing counts per cell
    finished (0:00:00)


In [18]:
adata.raw = adata.copy()

In [ ]:
sc.pp.highly_variable_genes(adata)
sc.pp.scale(adata, max_value=10)
sc.pp.pca(adata)

In [ ]:
adata.obs

In [ ]:
sce.pp.harmony_integrate(adata, 'Chemistry', adjusted_basis='X_pca', max_iter_harmony=50)

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)